In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
import nltk
import numpy as np
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')


/ihome/xli/sek188/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 n

True

In [10]:
# !pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [2]:
model_name = "mediabiasgroup/da-roberta-babe-ft"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)




In [5]:
#make dataset
wdf = pd.read_csv('combined_wide_form_jul30.csv')
wdf = wdf[['article_set_unique_id', 'source_url', 'title', 'text', 'human_clean_title', 'human_clean_text', 'pull']]
wdf['clean'] = 'HEADLINE: ' + wdf['human_clean_title'] + "ARTICLE" + wdf['human_clean_title']

In [6]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# text = "the left gets absolutely roasted by the super awesome gigachad trump" lable 1, score 0.938
# text = "the fed cuts interest rates"  label 0, score 0.95
result = classifier(text)

# print(result)

Device set to use cuda:0


NameError: name 'text' is not defined

In [7]:
def classify_article(text):
    # split into sentences
    sentences = sent_tokenize(text)

    # Step 2: Run classifier on each sentence
    try:
        results = classifier(sentences)
    
    
        # Step 3: Convert to array of scores (shape: [num_sentences, num_classes])
        if not isinstance(results, list):
            results = [results]
    
        score_array = np.array([r.get('score') for r in results])
        mean_scores = score_array.mean(axis=0)
    
        # Step 4: Average across sentences
        
    
        # Step 5: Final prediction
        adjusted_score = 0
        for r in results:
            if r.get('label', None) == 'LABEL_1':
                adjusted_score += r.get("score", 0)
            else:
                adjusted_score -= r.get("score", 0)
        # pred_index = int(np.argmax(mean_scores))
        # pred_label = results[0][pred_index]['label']
        pred_label = "LABEL_1" if adjusted_score > 0 else "LABEL_0"
    
        return {
            'label': pred_label,
            'score': mean_scores,
            'scores': score_array,
            'adjusted_score': adjusted_score
        }

    except Exception as e:
        print(results)
        print(e)
        return {
            'label': "error",
            'score': str(e),
            'scores': results
        }

In [8]:
# wdf = wdf.sample(2)
wdf['spinde_output'] = wdf['clean'].apply(classify_article)

wdf['spinde_label'] = wdf['spinde_output'].apply(lambda x: x['label'])
wdf['spinde_score'] = wdf['spinde_output'].apply(lambda x: x['score'])
wdf['spinde_scores'] = wdf['spinde_output'].apply(lambda x: x['scores'])


# [{'label': 'LABEL_0', 'score': 0.9084169268608093}, {'label': 'LABEL_0', 'score': 0.9547224640846252}]
# string indices must be integers
# [{'label': 'LABEL_0', 'score': 0.9597590565681458}]

# string indices must be integers

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [9]:
# wdf = wdf.reset_index()
# res = wdf.loc[0, 'spinde_output']

wdf.head()
wdf.to_csv('spinde_wide_form_jul30.csv')


In [10]:
wdf['spinde_label'].value_counts()

spinde_label
LABEL_0    338
LABEL_1     91
Name: count, dtype: int64